# 1. Graph Construction

In [ ]:
!pip install gensim
!pip install matplotlib
!pip install -U scikit-learn
!pip install torch_geometric
!pip install networkx
!pip3 install torch torchvision torchaudio
!pip install --no-index torch-sparse -f https://pytorch-geometric.com/whl/torch-2.0.1+cpu.html
!pip install --no-index torch-scatter -f https://pytorch-geometric.com/whl/torch-2.0.1+cpu.html
!pip install --no-index torch-cluster -f https://pytorch-geometric.com/whl/torch-2.0.1+cpu.html
!pip install --no-index torch-spline-conv -f https://pytorch-geometric.com/whl/torch-2.0.1+cpu.html

In [ ]:
import networkx as nx
from torch_geometric.datasets.planetoid import Planetoid
from torch_geometric.datasets.aminer import AMiner
from torch_geometric.utils import to_networkx, to_dense_adj

In [ ]:
# Cora Dataset for GraRep
cora_data = Planetoid(root='/tmp/Cora', name='Cora')[0]

In [ ]:
cora_data

In [ ]:
cora_A = to_dense_adj(cora_data.edge_index)[0]

In [ ]:
cora_A.shape

In [ ]:
# AMiner Dataset for Metapath2vec
!pip install pandas
aminer_data = AMiner("./")[0]

In [ ]:
aminer_data

# 2. An example of Computing Transition Matrix

In [ ]:
import networkx as nx
import numpy as np

from sklearn.preprocessing import normalize

nx_g = nx.Graph()

nx_g.add_edges_from([(0, 1), (0, 2),  (2, 3), (2, 4), (3,5 ), (0,3), (0,5), (1,5), (3,6), (2,5), (1,7),(8,7), (3,9)])
nx.draw(nx_g,with_labels = True)
num_nodes = 10

In [ ]:
Ak = np.matrix(np.identity(num_nodes))
adj = nx.adjacency_matrix(nx_g)
adj= adj.todense()
adj = normalize(adj, axis=1, norm='l1')
adj = np.round(adj,2)
adj

In [ ]:
for i in range(4):
    Ak = np.dot(Ak, adj)
    print(f'Step: {i+1}')
    print(np.round(Ak,2))

# 3. Grarep

In [ ]:
import math
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy import sparse
from sklearn.decomposition import TruncatedSVD

In [ ]:
class GraRep(object):
    def __init__(self, A, args):
        """
        :param A: Adjacency matrix.
        :param args: Arguments object.
        """
        self.A = A
        self.args = args
        self._setup_base_target_matrix()

    def _setup_base_target_matrix(self):
        """
        Creating a base matrix to multiply.
        """
        values = [1.0 for _ in range(self.A.shape[0])]
        indices = [i for i in range(self.A.shape[0])]
        self.A_hat = sparse.coo_matrix((values, (indices, indices)),
                                       shape=self.A.shape,
                                       dtype=np.float32)

    def _create_target_matrix(self):
        """
        Creating a log transformed target matrix.
        :return target_matrix: Matrix to decompose with SVD.
        """
        self.A_hat = sparse.coo_matrix(self.A_hat.dot(self.A))
        scores = np.log(self.A_hat.data)-math.log(self.A.shape[0])
        rows = self.A_hat.row[scores < 0]
        cols = self.A_hat.col[scores < 0]
        scores = scores[scores < 0]
        target_matrix = sparse.coo_matrix((scores, (rows, cols)),
                                          shape=self.A.shape,
                                          dtype=np.float32)
        return target_matrix

    def optimize(self):
        """
        Learning an embedding.
        """
        print("\nOptimization started.\n")
        self.embeddings = []
        for step in tqdm(range(self.args["order"])):
            target_matrix = self._create_target_matrix()

            svd = TruncatedSVD(n_components=self.args["dim"],
                               n_iter=self.args["iter"],
                               random_state=self.args["seed"])

            svd.fit(target_matrix)
            embedding = svd.transform(target_matrix)
            self.embeddings.append(embedding)

    def return_embedding(self):
        """
        Return the embedding.
        """
        print("\nReturn embedding.\n")
        self.embeddings = np.concatenate(self.embeddings, axis=1)
        column_count = self.args["order"] * self.args["dim"]
        columns = ["ID"] + ["x_" + str(col) for col in range(column_count)]
        ids = np.array([i for i in range(self.A.shape[0])]).reshape(-1,1)
        self.embeddings = np.concatenate([ids, self.embeddings], axis=1)
        self.embeddings = pd.DataFrame(self.embeddings, columns=columns)
        
        return self.embeddings

In [ ]:
args = {
    "dim": 16,
    "order": 5,
    "seed": 42,
    "iter": 20
}

grarep = GraRep(cora_A, args)

grarep.optimize()

In [ ]:
grarep_emb_csv = grarep.return_embedding()

In [ ]:
grarep_emb_csv.head()

# 4. SDNE

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [ ]:
class SDNE(nn.Module):
    def __init__(self, node_size, nhid0, nhid1, dropout, alpha):
        """
        :param node_size: the size of node
        :param nhid0: the input vector size
        :param nhid1: the output vector size
        :param dropout: the value for dropout layer
        :param alpha: the value for loss
        """
        super(SDNE, self).__init__()
        self.encode0 = nn.Linear(node_size, nhid0)
        self.encode1 = nn.Linear(nhid0, nhid1)
        self.decode0 = nn.Linear(nhid1, nhid0)
        self.decode1 = nn.Linear(nhid0, node_size)
        self.droput = dropout
        self.alpha = alpha

    def forward(self, adj_batch, adj_mat, b_mat):
        """
        :param adj_batch: [batch_size, the number of node] => adj matrix for node in batch
        :param adj_mat: [batch_size, batch_size] => adj matrixf for node only in batch
        :param b_mat: 
        :return: B matrix for L_2nd
        """
        t0 = F.leaky_relu(self.encode0(adj_batch))
        t0 = F.leaky_relu(self.encode1(t0))
        embedding = t0
        t0 = F.leaky_relu(self.decode0(t0))
        t0 = F.leaky_relu(self.decode1(t0))
        embedding_norm = torch.sum(embedding * embedding, dim=1, keepdim=True)
        L_1st = torch.sum(adj_mat * (embedding_norm -
                                     2 * torch.mm(embedding, torch.transpose(embedding, dim0=0, dim1=1))
                                     + torch.transpose(embedding_norm, dim0=0, dim1=1)))
        L_2nd = torch.sum(((adj_batch - t0) * b_mat) * ((adj_batch - t0) * b_mat))
        return L_1st, self.alpha * L_2nd, L_1st + self.alpha * L_2nd

    def savector(self, adj):
        t0 = self.encode0(adj)
        t0 = self.encode1(t0)
        return t0

In [ ]:
class SDNEDataset(Dataset):
    def __init__(self, Adj, Node):
        """
        Dataset Class for iteration
        
        :param Adj: adh matrix
        :param Node: the number of nodes
        """
        self.Adj = Adj
        self.Node = Node
    def __getitem__(self, index):
        '''
        
        :param index: the index of node
        :return: the index of node 
        '''
        return index

    def __len__(self):
        return self.Node

In [ ]:
config = {
    "in_dim": 1433,
    "out_dim": 128,
    "dropout": 0.5,
    "alpha": 1e-2,
    "lr": 0.001,
    "step_size": 10,
    "gamma": 0.9,
    "batch_size": 100,
    "epoch": 100,
    "beta": 5,
    "nu1": 1e-5,
    "nu2": 1e-4
}

G, Adj, Node =  to_networkx(cora_data), cora_A, cora_A.shape[0]
model = SDNE(Node, config["in_dim"], config["out_dim"], config["dropout"], config["alpha"])
opt = torch.optim.Adam(model.parameters(), lr=config["lr"])
scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=config["step_size"], gamma=config["gamma"])
Data = SDNEDataset(Adj, Node)
Data = DataLoader(Data, batch_size=config["batch_size"], shuffle=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.train()

iter_bar = tqdm(range(1, config["epoch"] + 1))
for epoch in iter_bar:
    loss_sum, loss_L1, loss_L2, loss_reg = 0, 0, 0, 0
    for index in Data:
        adj_batch = Adj[index].to(device)
        adj_mat = adj_batch[:, index].to(device)
        b_mat = torch.ones_like(adj_batch).to(device)
        b_mat[adj_batch != 0] = config["beta"]
        opt.zero_grad()
        L_1st, L_2nd, L_all = model(adj_batch, adj_mat, b_mat)
        L_reg = 0
        for param in model.parameters():
            L_reg += config["nu1"] * torch.sum(torch.abs(param)) + config["nu2"] * torch.sum(param * param)
        Loss = L_all + L_reg
        
        # back propagation
        Loss.backward()
        opt.step()
        loss_sum += Loss
        loss_L1 += L_1st
        loss_L2 += L_2nd
        loss_reg += L_reg
        
        # update progress
        scheduler.step(epoch)
        iter_bar.set_postfix({"loss_sum": loss_sum.item(), "loss_L1": loss_L1.item(), "loss_L2": loss_L2.item(), "loss_reg": loss_reg.item()})
        
model.eval()
embedding = model.savector(Adj.to(device))
outVec = embedding.cpu().detach().numpy()

In [ ]:
outVec

# 5. Metapath2vec

In [ ]:
import torch
from torch_geometric.nn.models import MetaPath2Vec

In [ ]:
metapath = [
    ('author', 'writes', 'paper'),
    ('paper', 'published_in', 'venue'),
    ('venue', 'publishes', 'paper'),
    ('paper', 'written_by', 'author'),
]

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = MetaPath2Vec(aminer_data.edge_index_dict, embedding_dim=128,
                     metapath=metapath, walk_length=50, context_size=7,
                     walks_per_node=5, num_negative_samples=5,
                     sparse=True).to(device)

loader = model.loader(batch_size=128, shuffle=True, num_workers=6)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)

In [ ]:
def train(epoch, log_steps=100, eval_steps=2000):
    '''
    :param log_steps: the value of step for log 
    :param eval_steps: the value of step for evaluation
    :return: 
    '''
    model.train()

    total_loss = 0
    for i, (pos_rw, neg_rw) in enumerate(loader):
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        if (i + 1) % log_steps == 0:
            print((f'Epoch: {epoch}, Step: {i + 1:05d}/{len(loader)}, '
                   f'Loss: {total_loss / log_steps:.4f}'))
            total_loss = 0

        if (i + 1) % eval_steps == 0:
            acc = test()
            print((f'Epoch: {epoch}, Step: {i + 1:05d}/{len(loader)}, '
                   f'Acc: {acc:.4f}'))


@torch.no_grad()
def test(train_ratio=0.1):
    model.eval()

    z = model('author', batch=aminer_data['author'].y_index.to(device))
    y = aminer_data['author'].y

    perm = torch.randperm(z.size(0))
    train_perm = perm[:int(z.size(0) * train_ratio)]
    test_perm = perm[int(z.size(0) * train_ratio):]

    return model.test(z[train_perm], y[train_perm], z[test_perm], y[test_perm],
                      max_iter=150)

In [ ]:
for epoch in range(5):
    train(epoch)
    acc = test()
    print(f'Epoch: {epoch}, Accuracy: {acc:.4f}')

# 6. HIN2Vec

In [ ]:
import random
from itertools import product
from collections import defaultdict

In [ ]:
def binary_reg(x: torch.Tensor):
    # forward: f(x) = (x>=0)
    # backward: f(x) = sigmoid
    a = torch.sigmoid(x)
    b = a.detach()
    c = (x.detach() >= 0).float()
    return a - b + c

class HIN2vec(nn.Module):

    def __init__(self, node_size, path_size, embed_dim, sigmoid_reg=False, r=True):
        '''
        
        :param node_size: the number of nodes
        :param path_size: the length of path
        :param embed_dim: the size of dim for embedding
        :param sigmoid_reg: the argument for setting sigmoid
        :param r: the argument for pretrained loss (if True, model will be trained with random embedding table)
        '''
        super().__init__()

        self.reg = torch.sigmoid if sigmoid_reg else binary_reg

        self.__initialize_model(node_size, path_size, embed_dim, r)

    def __initialize_model(self, node_size, path_size, embed_dim, r):
        self.start_embeds = nn.Embedding(node_size, embed_dim)
        self.end_embeds = self.start_embeds if r else nn.Embedding(node_size, embed_dim)

        self.path_embeds = nn.Embedding(path_size, embed_dim)
        # self.classifier = nn.Sequential(
        #     nn.Linear(embed_dim, 1),
        #     nn.Sigmoid(),
        # )

    def forward(self, start_node: torch.LongTensor, end_node: torch.LongTensor, path: torch.LongTensor):
        '''
        
        :param start_node: the input of start node
        :param end_node:  the input of end node
        :param path:  the input of path
        :return: the output of model
        '''
        # assert start_node.dim() == 1  # shape = (batch_size,)

        s = self.start_embeds(start_node)  # (batch_size, embed_size)
        e = self.end_embeds(end_node)
        p = self.path_embeds(path)
        p = self.reg(p)

        agg = torch.mul(s, e)
        agg = torch.mul(agg, p)
        # agg = F.sigmoid(agg)
        # output = self.classifier(agg)

        output = torch.sigmoid(torch.sum(agg, axis=1))

        return output


def train(log_interval, model, device, train_loader: DataLoader, optimizer, loss_function, epoch):
    model.to(device)
    model.train()
    for idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data[:, 0], data[:, 1], data[:, 2])
        loss = loss_function(output.view(-1), target)
        loss.backward()
        optimizer.step()

        if idx % log_interval == 0:
            print(f'\rTrain Epoch: {epoch} '
                  f'[{idx * len(data)}/{len(train_loader.dataset)} ({100. * idx / len(train_loader):.3f}%)]\t'
                  f'Loss: {loss.item():.3f}\t\t',
                  # f'data = {data}\t target = {target}',
                  end='')
    print()


class NSTrainSet(Dataset):
    def __init__(self, sample, node_size, neg=5):
        """
        :param node_size: the number of nodes
        :param neg: the number of negative samples
        :param sample: return value of HIN.sample()，(start_node, end_node, path_id)
        """

        print('init training dataset...')

        l = len(sample)

        x = np.tile(sample, (neg + 1, 1))
        y = np.zeros(l * (1 + neg))
        y[:l] = 1

        # x[l:, 2] = np.random.randint(0, path_size - 1, (l * neg,))
        x[l:, 1] = np.random.randint(0, node_size - 1, (l * neg,))

        self.x = torch.LongTensor(x)
        self.y = torch.FloatTensor(y)
        self.length = len(x)

        print('finished')

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return self.length

In [ ]:
class HIN:
    """
    Class to generate vertex sequences.
    """
    def __init__(self, window=None):
        self.graph = nx.DiGraph()
        self.node_size = 0
        self._path_size = 0

        def new_id():
            i = self.node_size
            self.node_size += 1
            return i

        self._node2id = defaultdict(new_id)
        self._id2type = {}
        self._window = window
        self._node_types = set()
        self._path2id = None
        self._id2path = None
        self._id2node = None

    @property
    def id2node(self):
        return self._id2node

    @property
    def id2path(self):
        return self._id2path

    @property
    def window(self):
        return self._window

    @window.setter
    def window(self, val):
        if not self._window:
            self._window = val
        else:
            raise ValueError("window error")

    @property
    def path_size(self):
        if not self._path_size:
            raise ValueError("run sample() first to count path size")
        return self._path_size

    def add_edge(self, source_node, source_class, dest_node, dest_class, edge_class, weight):
        i = self._node2id[source_node]
        j = self._node2id[dest_node]
        self._id2type[i] = source_class
        self._id2type[j] = dest_class
        self._node_types.add(source_class)
        self._node_types.add(dest_class)
        self.graph.add_edge(i, j, weight=weight)

    def small_walk(self, start_node, length):
        walk = [start_node]
        for i in range(1, length):
            if next(nx.neighbors(self.graph, walk[-1]), None) is None:
                break
            cur_node = walk[-1]
            nodes = list(nx.neighbors(self.graph, cur_node))
            weights = [self.graph[cur_node][i]['weight'] for i in nodes]
            s = sum(weights)
            weights = [i/s for i in weights]
            walk += random.choices(nodes, weights, k=1)
        return walk

    def do_walks(self, length):
        for start_node in range(self.node_size):
            yield self.small_walk(start_node, length)

    def sample(self, length, n_repeat):
        """
        :param length: the length of path
        :param n_repeat: the number of repeat
        :return: (start_id, end_id, edge_type)
        """
        if not self.window:
            raise ValueError("window not set")

        if not self._path2id:
            self._path2id = {}
            path_id = 0
            for w in range(1, self._window + 1):
                for i in product(self._node_types, repeat=w + 1):
                    self._path2id[i] = path_id
                    path_id += 1

            self._path_size = len(self._path2id)
            self._id2node = {v: k for k, v in self._node2id.items()}
            self._id2path = {v: k for k, v in self._path2id.items()}

        samples = []

        for repeat in range(n_repeat):
            for walk in self.do_walks(length):
                cur_len = 0
                for i, node_id in enumerate(walk):
                    cur_len = min(cur_len + 1, self._window + 1)  # 当window=n的时候，最长路径有n+1个节点
                    if cur_len >= 2:
                        for path_length in range(1, cur_len):
                            sample = (walk[i - path_length], walk[i],
                                      self._path2id[tuple([self._id2type[t] for t in walk[i - path_length:i + 1]])])
                            # print(tuple([self.id2type[t] for t in walk[i-path_length:i + 1]]))
                            samples.append(sample)

        return samples

    def print_statistics(self):
        print(f'size = {self.node_size}')


def load_a_HIN_from_pandas(edges, print_graph=False):
    """
    edges = list(pd.df)
    """

    def reverse(df):
        """
        reverse source & dest
        """
        df = df.rename({'source_node': 'dest_node', 'dest_node': 'source_node',
                        'source_class': 'dest_class', 'dest_class': 'source_class'},
                       axis=1)
        # reverse edge_class
        df.edge_class = df.edge_class.map(lambda x: '-'.join(reversed(x.split('-'))))
        return df

    print('load graph from edges...')
    g = HIN()
    if isinstance(edges, list):
        edges = pd.concat(edges, sort=False)
    # edges = edges.append(reverse(edges), sort=False, ignore_index=True)
    edges = pd.concat([edges, reverse(edges)], ignore_index=True, sort=False)

    for index, row in edges.iterrows():
        g.add_edge(row['source_node'], row['source_class'],
                   row['dest_node'], row['dest_class'], row['edge_class'],
                   row['weight'])
    if print_graph:
        g.print_statistics()
    print('finish loading graph!')
    return g

In [ ]:
# set method parameters
window = 4
walk = 10
walk_length = 300
embed_size = 100
neg = 5
sigmoid_reg = True 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'device = {device}')

# set dataset [PLEASE USE YOUR OWN DATASET TO REPLACE THIS]
demo_edge = pd.read_csv('./sample_graph_df.csv', index_col=0)

In [ ]:
demo_edge

In [ ]:
edges = [demo_edge]

print('finish loading edges')

# init HIN
hin = load_a_HIN_from_pandas(edges)
hin.window = window

dataset = NSTrainSet(hin.sample(walk_length, walk), hin.node_size, neg=neg)

hin2vec = HIN2vec(hin.node_size, hin.path_size, embed_size, sigmoid_reg)

In [ ]:
# set training parameters
n_epoch = 1
batch_size = 20
log_interval = 200

data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
optimizer = torch.optim.AdamW(hin2vec.parameters())
loss_function = nn.BCELoss()

for epoch in range(n_epoch):
    train(log_interval, hin2vec, device, data_loader, optimizer, loss_function, epoch)

torch.save(hin2vec, 'hin2vec.pt')

In [ ]:
# set output parameters [the output file is a bit different from the original code.]
node_vec_fname = 'node_vec.txt'
path_vec_fname = "path_2vec.txt"
    
print(f'saving node embedding vectors to {node_vec_fname}...')
node_embeds = pd.DataFrame(hin2vec.start_embeds.weight.data.cpu().numpy())
node_embeds.rename(hin.id2node).to_csv(node_vec_fname, sep=' ')

In [ ]:
node_embeds

# 7. WL-relabeling

In [ ]:
def wlk_relabel(g,h):
    '''
    Perform node relabeling (coloring) according 1-d WL relabeling process (refer Shervashidze et al (2009) paper)
    :param g: networkx graph
    :param h: height of WL kernel
    :return: relabeled graph
    '''
    for i in range(len(g.nodes)):
        g.nodes[i]['relabel'] = {}
        
    for i in range(0,h+1): #xrange returns [min,max)
        for n in range(len(g.nodes)):
            # degree_prefix = 'D' + str(i)
            degree_prefix = ''
            if 0 == i:
                g.nodes[n]['relabel'][0] = degree_prefix + str(g.nodes[n]['label']).strip() + degree_prefix
            else:
                nei_labels = [g.nodes[nei]['relabel'][i-1] for nei in nx.all_neighbors(g,n)]
                nei_labels.sort()
                sorted_nei_labels = (','*i).join(nei_labels)

                current_in_relabel = g.nodes[n]['relabel'][i-1] +'#'*i+ sorted_nei_labels
                g.nodes[n]['relabel'][i] = degree_prefix + current_in_relabel.strip() + degree_prefix
    return g #relabled graph

In [ ]:
def fill_labels(G):
    for i in range(len(G.nodes)):
        G.nodes[i]['label'] = '1'

In [ ]:
def draw_graph(G):
    clubs = []  # list to populate with labels
    for n in G.nodes:
        c = G.nodes[n]['relabel']
        print(f'Node ID {n}: colour: {c}')
        col = 0
        if c=='1':
            clubs.append('green')
        elif c =='2':
            clubs.append('red')
        else:
            clubs.append('gray')
    pos = nx.spring_layout(G, seed=42) # To be able to recreate the graph layout
    nx.draw_networkx(G, pos=pos, node_color = clubs) # Plot the graph

In [ ]:
G = nx.Graph([
    (0, 1),
    (1 ,2), 
    (1 ,3),
    (1 ,5),
    (2 ,3),
    (2 ,4),
    (2 ,6),
    (3 ,4),
    (3 ,5),
    (4 ,7),
    (4 ,8),
    (6 ,7)])
fill_labels(G)
nx.draw(G, with_labels = True)

In [ ]:
relabeled_G = wlk_relabel(G, 1)

In [ ]:
draw_graph(relabeled_G)